In [1]:
! pip install sentence_transformers rank_bm25 cohere

  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import cohere


In [ ]:
docunents = [
"This is a list which containing sample documents.",
"Keywords are important for keyword-based search.",
"Document analysis involves extracting keywords.",
"Keyword-based search relies on sparse embeddings.",
"Understanding document structure aids in keyword extraction.",
"Efficient keyword extraction enhances search accuracy.",
"Semantic similarity improves document retrieval performance.",
"Machine learning algorithms can optimize keyword extraction methods."
]

In [ ]:
# Load pre-traind sentence transformer model
model_name = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

In [ ]:
model = SentenceTransformer(model_name)

In [ ]:
docunents_embeddings = model.encode(docunents)

In [ ]:
for i, embedding in enumerate(docunents_embeddings):
    print(f"Documents {i + 1} embedding: {embedding}")

In [ ]:
query = "Natural Language Processing techniques enhance keyword extraction efficiency"

In [ ]:
query_embedding = model.encode(query)

In [ ]:
print("Query embedding: ", query_embedding)

In [ ]:
similarities = cosine_similarity(np.array([query_embedding]), docunents_embeddings)

In [ ]:
similarities

In [ ]:
most_similar_index = np.argmax(similarities)

In [ ]:
most_similar_index

In [ ]:
most_similar_document = docunents[most_similar_index]

In [ ]:
most_similar_document

In [ ]:
similarity_score = similarities[0][most_similar_index]

In [ ]:
similarity_score

In [ ]:
sorted_indices = np.argsort(similarities[0])[::-1]

In [ ]:
sorted_indices

In [ ]:
ranked_documents = [(docunents[i], similarities[0][i]) for i in sorted_indices]

In [ ]:
ranked_documents

In [ ]:
print("Ranked Documents: ")
for rank, (document, similarity) in enumerate(ranked_documents, start=1):
    print(f"Rank: {rank}, Document: {document}, Similarity: {similarity}")

In [ ]:
print("Top 4 Documents: ")

for rank, (document, similarity) in enumerate(ranked_documents[:4], start=1):
    print(f"Rank: {rank}, Documents: {document}, Similarity_score: {similarity}")

In [ ]:
top_4_documents = [doc[0] for doc in ranked_documents[:4]]

In [ ]:
tokenized_top_4_documents = [doc.split() for doc in top_4_documents]

In [ ]:
tokenized_top_4_documents

In [ ]:
tokenized_query = query.split()

In [ ]:
tokenized_query

In [ ]:
bm25 = BM250kapi(tokenized_top_4_documents)

In [ ]:
bm25_scores = bm25.get_scores(tokenized_query)

In [ ]:
bm25_scores

In [ ]:
sorted_indices_2 = np.argsort(bm25_scores)[::-1]

In [ ]:
sorted_indices_2

In [ ]:
reranked_documents = [(top_4_documents[i], bm25_scores[i]) for i in sorted_indices_2]

In [ ]:
reranked_documents

In [ ]:
print("Top 4 Documents: ")

for rank, (document, similarity) in enumerate(reranked_documents, start=1):
    print(f"Rank: {rank}, Documents: {document}, Similarity_score: {similarity}")

In [ ]:
reranked_documents[:4]

Cross encoder

In [ ]:
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-5-v2")

In [ ]:
pairs = []

for doc in top_4_documents:
    pairs.append([query, doc])

In [ ]:
pairs

In [ ]:
scores = cross_encoder.predict(pairs)
scores

In [ ]:
scored_docs = zip(scores, top_4_documents)
scored_docs

In [ ]:
reranked_document_cross_encoder = sorted(scored_docs, reverse=True)

In [ ]:
reranked_document_cross_encoder

BM_25

In [ ]:
reranked_documents

In [ ]:
co = cohere.Client("COHERE_API_KEY")

In [ ]:
response = co.rerank(
    model="rerank-english-v3.0",
    query="Natural language processing techniques enhance keyword extraction efficiency ",
    documents=top_4_documents,
    return_documents=True
)

In [ ]:
print(response)

In [ ]:
response.results[0].document.text

In [ ]:
response.results[0].relevance_score

In [ ]:
for i in range(4):
    print(f"Text: {response.results[i].document.text}, Score: {response.results[i].relevance_score}")

In [ ]:
# Initialize model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
sentence_a = "The movie was fantastic"
sentence_b = "I really enjoyed the film"

In [ ]:
# Encode Input
inputs = tokenizer(sentence_a, sentence_b, return_tensors="pt", padding=True, truncation=True)

In [ ]:
# Get model output
outputs = model(**inputs)
logits = outputs.logits 

In [ ]:
# Convert logits to probabilities
probs = torch.softmax(logits, dim=1)

In [ ]:
# Assuming a binary classification task (similarity score)
similarity_score_1 = probs[0][1].item()
print(f"Similarity Score: {similarity_score_1}")